In [105]:
import pandas as pd
import psycopg2
import re

def df_nan_filter(df):
    df = df[df["Size"].notnull()]              # remove "Size" row if size is null
    df["Languages"].fillna("EN", inplace=True) # set "EN" as langage if value is null
    df["Price"].fillna(0.0, inplace=True)      # set 0.0 as price if value is null
    df["Average User Rating"].fillna(df["Average User Rating"].median(), inplace=True) # set median as user_rating if value is null
    df["User Rating Count"].fillna(1, inplace=True) #replace nan counts with 1
    return (df)

def special_char_filter(s: str):
    s = re.sub(r'\\u.{4}', '', s)
    s = re.sub(r'\\x.{2}', '', s)
    s = re.sub(r'\\n', ' ', s)
    s = re.sub(r'\\t', ' ', s)
    s = re.sub(r'\s+', ' ', s)
    s = s.strip('"')
    return (''.join(e for e in s if e.isascii()))

df = pd.read_csv("appstore_games.csv")
df = df[["ID", "Name", "Average User Rating",
          "User Rating Count", "Price", "Description",
          "Developer", "Age Rating", "Languages",
          "Size", "Primary Genre", "Genres",
          "Original Release Date", "Current Version Release Date"]]

df = df_nan_filter(df)

for col in df:
    df[col] = df[col].apply(lambda x: special_char_filter(str(x)))

df.to_csv("filtered_appstore_games.csv")

In [2]:
def already_exists():
    conn = psycopg2.connect(
        database="appstore_games",
        host="localhost",
        user="postgres_user",
        password="12345"
    )
    curr = conn.cursor()
    curr.execute("select exists(select * from information_schema.tables where table_name=%s)", ('appstore_games',))         
    ver = curr.fetchone()[0]
    conn.close()
    return (ver)

def create_table():
    conn = psycopg2.connect(
        database="appstore_games",
        host="localhost",
        user="postgres_user",
        password="12345"
    )
    curr = conn.cursor()
    curr.execute("""CREATE TABLE appstore_games(
             Id bigint PRIMARY KEY,
             Name varchar,
             Avg_user_rating float,
             User_rating_count int,
             Price float,
             Description varchar,
             Developer varchar,
             Age_rating int,
             Languages varchar,
             Size bigint, 
             Primary_genre varchar,
             Genres varchar,
             Release_date date,
             Last_update date
             );""")          
    conn.commit()
    conn.close()

def delete_table():
    conn = psycopg2.connect(
        database="appstore_games",
        host="localhost",
        user="postgres_user",
        password="12345"
    )
    curr = conn.cursor()
    curr.execute('DROP TABLE appstore_games;')          
    conn.commit()
    conn.close()

In [5]:
create_table()
#delete_table()

In [7]:
import pandas as pd

df = pd.read_csv("filtered_appstore_games.csv")

In [32]:
from psycopg2 import sql

def insert_table():
    conn = psycopg2.connect(
        database="appstore_games",
        host="localhost",
        user="postgres_user",
        password="12345"
    )
    curr = conn.cursor()
    for idx in range(df.shape[0]):
        tmp = tuple(df.iloc[idx][1:])
        query = """INSERT INTO %s (Id, Name, Avg_user_rating, User_rating_count, Price, Description,
                Developer, Age_rating, Languages, Size, Primary_genre, Genres, Release_date, Last_update
              ) VALUES (%%s, %%s, %%s, %%s, %%s, %%s, %%s, %%s, %%s, %%s, %%s, %%s, %%s, %%s)""" % "appstore_games"
        #curr.execute(query, tmp)
        print(tmp)
        break
    conn.commit()
    conn.close()
insert_table()

(284921427, 'Sudoku', 4.0, 3553.0, 2.99, 'Join over 21,000,000 of our fans and download one of our Sudoku games today!\\n\\nMakers of the Best Sudoku Game of 2008, Sudoku (Free), we offer you the best selling Sudoku game for iPhone with great features and 1000 unique puzzles! \\n\\nSudoku will give you many hours of fun and puzzle solving. Enjoy the challenge of solving Sudoku puzzles whenever or wherever you are using your iPhone or iPod Touch. \\n\\nOPTIONS\\n\\nAll options are on by default, but you can turn them off in the Options menu\\n\\u2022 Show Incorrect :: Shows incorrect answers in red.  \\n\\u2022 Smart Buttons :: Disables the number button when that number is completed on the game board.   \\n\\u2022 Smart Notes :: Removes the number from the notes in the box, column, and row that contains the cell with your correct answer.\\n\\nFEATURES\\n\\n\\u2022 1000 unique handcrafted puzzles\\n\\u2022 ALL puzzles solvable WITHOUT guessing\\n\\u2022 Four different skill levels\\n\\u